In [1]:
import pandas as pd

In [2]:
import sqlite3

In [3]:
conn = sqlite3.connect('tennis_data.db')

In [4]:
shelton_query = """
SELECT * 
FROM matches
WHERE (Player_1 = 'Shelton B.' OR Player_2 = 'Shelton B.')
"""

In [5]:
df_shelton = pd.read_sql(shelton_query, conn)

In [6]:
conn.close()

In [7]:
df_shelton['Shelton_Won'] = (df_shelton['Winner'] == 'Shelton B.').astype(int)

In [8]:
surface_stats = df_shelton.groupby('Surface')['Shelton_Won'].agg(['count', 'sum', 'mean'])

In [9]:
surface_stats.columns = ['Total Matches', 'Wins', 'Win Rate']

In [10]:
surface_stats['Win Rate'] = surface_stats['Win Rate'] * 100

display(surface_stats)

,Total Matches,Wins,Win Rate
Surface,,,
Clay,32,16,50.000000
Grass,23,12,52.173913
Hard,99,60,60.606061


In [11]:
# --- STEP 1: AUTNHETIFICATION ---

import gspread
from google.oauth2.service_account import Credentials
import pandas as pd

scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

credentials = Credentials.from_service_account_file(
    'google.json',
    scopes=scopes
)

gc = gspread.authorize(credentials)

In [12]:
# --- STEP 2: SELECT TABLE ---
spreadsheet = gc.open('Tennis Reports')

worksheet = spreadsheet.sheet1 

In [13]:
# --- STEP 3: UPLOADING DATA ---
worksheet.clear()

final_report = surface_stats.reset_index()

headers = final_report.columns.values.tolist()

values = final_report.values.tolist()

data_to_upload = [headers] + values

worksheet.update('A1', data_to_upload)

C:\Temp\ipykernel_16092\50070434.py:12: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', data_to_upload)


{'spreadsheetId': '1MBGrd2kTAVhTbL0XvQpJxchrSXmIdxlkUlrHqmbRHAE',
 'updatedRange': 'Sheet1!A1:D4',
 'updatedRows': 4,
 'updatedColumns': 4,
 'updatedCells': 16}

In [14]:
print("Report successufully uploaded to Google Sheets!")
print(f"Link to the report: https://docs.google.com/spreadsheets/d/{spreadsheet.id}")


Report successufully uploaded to Google Sheets!
Link to the report: https://docs.google.com/spreadsheets/d/1MBGrd2kTAVhTbL0XvQpJxchrSXmIdxlkUlrHqmbRHAE
